### Default Values

We can specify **default** values for positional arguments.

This essentially makes the argument **optional** when we call the function - if we do not specify it, the default value will be used instead.

In [1]:
def func(a=1):
    return a

In [2]:
func()

1

In [3]:
func(10)

10

We can still used named (keyword) arguments:

In [4]:
func(a=10)

10

Let's expand on this a bit:

In [5]:
def func(a, b=10, c=20):
    return a, b, c

In [6]:
func(1)

(1, 10, 20)

In [7]:
func(1, 2)

(1, 2, 20)

In [8]:
func(1, 2, 3)

(1, 2, 3)

And we can mix positional and named (keyword) arguments too:

In [9]:
func(1, c=100)

(1, 10, 100)

As you can see, the first argument was positional, so it was assigned to `a`, and then we specified `c`, which means `b` used the default.

Let's see a practical application of this:

Suppose we want to compare two floats, and consider them equal if they differ by some small amount only (an absolute tolerance):

In [10]:
def is_close(a, b, abs_tol=0.01):
    return abs(a - b) <= abs_tol

Then we can use the default:

In [11]:
is_close(1.255, 1.256)

True

In [12]:
is_close(10_001, 10_002)

False

So for a large number like this, we may want to override the default tolerance value:

In [13]:
is_close(10_001, 10_002, 5)

True

Another example of this might be to split a string based on some separator and return some processed output:

In [14]:
def parse(s, sep=',', strip=True):
    items = s.split(sep)
    if strip:
        return [item.strip() for item in items]
    else:
        return items

In [15]:
parse('  a,   b ,  c  ')

['a', 'b', 'c']

Then we could tweak things as needed:

In [16]:
parse('a  :  b : c ', sep=':')

['a', 'b', 'c']

In [17]:
parse('a\n|b\n|c\n', sep='|')

['a', 'b', 'c']

In [18]:
parse('a\n|b\n|c\n', sep='|', strip=False)

['a\n', 'b\n', 'c\n']

Or maybe we want to generate a CSV style output from a list of lists:

In [19]:
data = [
    [10, 20, 30],
    [100, 200, 300],
    [1000, 2000, 3000]
]

In [20]:
def process_data(data, item_sep=',', line_sep='\n'):
    output = ''
    
    for row in data:
        for element in row:
            output = output + str(element) + item_sep
        output = output + line_sep
        
    return output

In [21]:
print(process_data(data))

10,20,30,
100,200,300,
1000,2000,3000,



Hmm... This kind of works, but there are some issues:

1. we have a trailing comma after each line
2. we have a blank line at the bottom (because the last data row also has that newline character)
3. we are using string concatenation - strings are immutable, so we are constantly creating new strings as we build our final output - not very efficient

Let's deal with the first problem - we can use `join` instead - this will take care of not appending that trailing comma.

Let's recall how `join` works:

In [22]:
'-'.join(['a', 'b', 'c'])

'a-b-c'

In [23]:
def process_data(data, item_sep=',', line_sep='\n'):
    output = ''
    
    for row in data:
        row_str = item_sep.join(row)
        output = output + row_str + line_sep
    return output    

In [24]:
print(process_data(data))

TypeError: sequence item 0: expected str instance, int found

So now we have another problem, our data contains integers, not strings - so we cannot use `join` - we have to convert our integers to strings first:

In [25]:
def process_data(data, item_sep=',', line_sep='\n'):
    output = ''
    
    for row in data:
        row_str = item_sep.join([str(element) for element in row])
        output = output + row_str + line_sep
    return output    

In [26]:
print(process_data(data))

10,20,30
100,200,300
1000,2000,3000



Better, but you'll notice we still have that blank row (so the last line of data has a trailing `line_sep`), and we're still using string concatenation.

Let's use `join` again, with another comprehension:

In [27]:
def process_data(data, item_sep=',', line_sep='\n'):
    row_strings = [item_sep.join([str(element) for element in row])
                  for row in data]
    return line_sep.join(row_strings)

In [28]:
print(process_data(data))

10,20,30
100,200,300
1000,2000,3000


Now we have something that works correctly, and is far more efficient than the first approach using a loop and string concatenation.

If this code looks a bit difficult to read, we could consider breaking this up (decomposition), to make it more readable:

In [29]:
def process_row(row, item_sep):
    return item_sep.join(str(element) for element in row)

In [30]:
def process_data(data, item_sep=',', line_sep='\n'):
    row_strings = [process_row(row, item_sep) for row in data]
    return line_sep.join(row_strings)

In [31]:
print(process_data(data))

10,20,30
100,200,300
1000,2000,3000


And now we can also specify different item and line separators:

In [32]:
print(process_data(data, '|'))

10|20|30
100|200|300
1000|2000|3000


In [33]:
print(process_data(data, line_sep='::'))

10,20,30::100,200,300::1000,2000,3000


In [34]:
print(process_data(data, item_sep='|', line_sep='\n\n'))

10|20|30

100|200|300

1000|2000|3000


One last tweak we can make is to use a generator instead of a list comprehension for `row_strings`:

In [35]:
def process_data(data, item_sep=',', line_sep='\n'):
    row_strings = (process_row(row, item_sep) for row in data)
    return line_sep.join(row_strings)

or even, though I think I prefer the above code more:

In [36]:
def process_data(data, item_sep=',', line_sep='\n'):
    return line_sep.join(process_row(row, item_sep) for row in data)

In [37]:
print(process_data(data))

10,20,30
100,200,300
1000,2000,3000
